# Student Loan Risk with Deep Learning

In [87]:
# Imports
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [63]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [64]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [65]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [66]:
# Define the target set y using the credit_ranking column

y = loans_df["credit_ranking"]

# Display a sample of y

y.values



array([0, 0, 0, ..., 1, 0, 1])

In [67]:
# Define features set X by selecting all columns but credit_ranking

X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame

X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [68]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [71]:
# Create a StandardScaler instance
scaler = StandardScaler()
scaler.fit(X_train)


# Fit the scaler to the features training dataset
# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [73]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [74]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
output_nodes = 7

In [75]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu", input_dim=input_nodes))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [76]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 3)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [77]:
# Compile the Sequential model
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])



In [78]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - accuracy: 0.4786 - loss: 0.6941 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 478us/step - accuracy: 0.5713 - loss: 0.6861
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 432us/step - accuracy: 0.6235 - loss: 0.6803
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step - accuracy: 0.6206 - loss: 0.6759
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.6443 - loss: 0.6614
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 404us/step - accuracy: 0.6747 - loss: 0.6502
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 390us/step - accuracy: 0.7099 - loss: 0.6273
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step - accuracy: 0.7110 - loss: 0.6066
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.6993 - loss: 0.6021
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step - accuracy: 0.7187 - loss: 0.5851
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.7374 - loss: 0.5581
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/ste

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [79]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 5ms/step - accuracy: 0.7475 - loss: 0.5160
Loss: 0.5159921050071716, Accuracy: 0.7475000023841858


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [42]:
# Set the model's file path
file_path = Path('student_loans.keras')

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [80]:
# Set the model's file path
file_path = Path('student_loans.keras')

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [81]:
# Make predictions with the test data
predictions = nn_model.predict(X_test_scaled,verbose=2)


# Display a sample of the predictions
predictions

13/13 - 0s - 3ms/step


array([[0.52519685],
       [0.34048188],
       [0.70998853],
       [0.7133587 ],
       [0.95485246],
       [0.8844634 ],
       [0.93230516],
       [0.20766954],
       [0.6769534 ],
       [0.28289187],
       [0.83974147],
       [0.26855823],
       [0.5646381 ],
       [0.8610344 ],
       [0.5259478 ],
       [0.20833768],
       [0.8579555 ],
       [0.3037177 ],
       [0.43659243],
       [0.4616402 ],
       [0.57273024],
       [0.84437066],
       [0.26401815],
       [0.94167626],
       [0.34952825],
       [0.84627384],
       [0.8571294 ],
       [0.64344674],
       [0.26511165],
       [0.24509019],
       [0.4655808 ],
       [0.97134984],
       [0.24961412],
       [0.94010013],
       [0.2809668 ],
       [0.64517075],
       [0.16662137],
       [0.32901052],
       [0.83928674],
       [0.2223544 ],
       [0.8400142 ],
       [0.07637708],
       [0.17251313],
       [0.89078313],
       [0.08770829],
       [0.75143415],
       [0.25069287],
       [0.247

In [82]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df["predictions"],0)
predictions_df

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [84]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       188
           1       0.78      0.73      0.75       212

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400



In [85]:
loans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   payment_history         1599 non-null   float64
 1   location_parameter      1599 non-null   float64
 2   stem_degree_score       1599 non-null   float64
 3   gpa_ranking             1599 non-null   float64
 4   alumni_success          1599 non-null   float64
 5   study_major_code        1599 non-null   float64
 6   time_to_completion      1599 non-null   float64
 7   finance_workshop_score  1599 non-null   float64
 8   cohort_ranking          1599 non-null   float64
 9   total_loan_score        1599 non-null   float64
 10  financial_aid_score     1599 non-null   float64
 11  credit_ranking          1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

 - This model will need data in three main categories: 
    - Student Financial (payment history, credit rating/credit score, financial aid score, finance workshop score): This would be the most critical data to predict ability to pay. 
    - Student Academic Profile (stem degree score, gpa, ranking, time to completion): This data could help to provide a more complete picture- certain fields of study may have higher earnings potential, especially with STEM. Academic performance could also be a predictor of repayment behavior. 
    - Student Demographic (location): understanding cost-of-living in the student's area can also be relevant to loan decisions.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

- This model would primarily use content-based filtering. The data collected (credit ranking, GPA, financial aid, major) provides specific traits that can be matched with appropriate loan options. There are some context-based features such as alumni success and financial workshop scores that could contribute to the model's performance, but not enough for context-based filtering to be the primary.  

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

- If a loan decision is made uweighting too heavily on academic profile as part of the data, it could present a risk. There could be a potential lag in the student academic profile if circumstances change, such as a change of major, withdrawal/drops. There is also a dependency on the institution to provide data, which can add complexity.

- There are potential risks of bias with the model. Students from wealthier areas might receive better loan terms due to higher credit rankings or financial aid scores. This could lead to inequities where students from disadvantaged backgrounds receive less favorable recommendations.